In [1]:
from lusidtools.jupyter_tools import toggle_code

"""Luminesce Syntax

Attributes
----------

"""

toggle_code("Toggle Docstring")


*Setting up the environment for the tutorial*

This section of the notebook creates a portfolio, some equities, and a file in drive to run luminesce queries against.

In [2]:
from lusid.utilities import ApiClientFactory
from lusidjam import RefreshingToken
import lusid as lu
import pandas as pd
import lusid_drive
import lusid_drive.rest
import lusid.rest
from lusid_drive.rest import ApiException
from pprint import pprint
import json


secrets_path = os.getenv("FBN_SECRETS_PATH")

api_factory = ApiClientFactory(
    token = RefreshingToken(), 
    api_secrets_filename = secrets_path,
    app_name = "LusidJupyterNotebook"
)

# Confirm success by printing SDK version
api_status = pd.DataFrame(api_factory.build(lu.ApplicationMetadataApi).get_lusid_versions().to_dict())
display(api_status)

,api_version,build_version,excel_version,links
0,v0,0.6.10288.0,0.5.3084,"{'relation': 'RequestLogs', 'href': 'http://fb..."


In [3]:
# here's some sample equities we've prepared earlier
uk_equity_instruments = pd.read_csv('_data/uk_equity_instruments.csv')
us_equity_instruments = pd.read_csv('_data/us_equity_instruments.csv')

In [4]:
# don't show exception if error is due to upsert
def exception_guard(e, code):
    return e.status and e.status != '400 Bad Request' and e.body and json.loads(e.body)["code"] == code

scope = 'FBNUniversityModule-T2-2' # str | The scope of the portfolio.

In [5]:
# create a request to upsert a bunch of instruments.
def generate_upsert_instrument_request(api_instance, name, ccy, figi, isin, other_identifier_name, other_identifier):
    request_id = name + isin
    request ={"name":name,
                     "identifiers":{ "Figi":{"value":figi},
                                     "Isin":{"value":isin},
                                     other_identifier_name:{"value":other_identifier}
                                   },

                     "definition":{"instrumentType":"Equity", "domCcy":ccy}}
    return request_id, request

In [6]:
# Create an instance of the API class
api_instance = api_factory.build(lu.InstrumentsApi)

# We're going to load some american and UK equities into LUSID

other_identifier_name = 'Sedol'
ccy = 'GBP'
upsert_body_tuple = (generate_upsert_instrument_request(api_instance, name, ccy, figi, isin, other_identifier_name, other_identifier)
               for 
               index, ticker, name, _, isin, other_identifier, figi
               in uk_equity_instruments.itertuples())
uk_equity_body_dict = {request_id:request_body for request_id,request_body in upsert_body_tuple}
other_identifier_name = 'Cusip'
ccy = 'USD'
upsert_body_tuple = (generate_upsert_instrument_request(api_instance, name, ccy, figi, isin, other_identifier_name, other_identifier)
               for 
               index, ticker, name, _, isin, other_identifier, figi
               in us_equity_instruments.itertuples())
us_equity_body_dict = {request_id:request_body for request_id,request_body in upsert_body_tuple}

request_body_dict = us_equity_body_dict | uk_equity_body_dict

try:
    # UpsertInstruments: Upsert instruments
    api_response = api_instance.upsert_instruments(request_body_dict, scope=scope)
#     pprint(api_response)
except lusid.rest.ApiException as e:
    print("Exception when calling InstrumentsApi->upsert_instruments: %s\n" % e)


In [7]:

# let's chuck a file containing a randomly selected subset of 25 instruments with their sectors into drive for some examples:

subset_file = open('_data/instrument_subset.csv').read()

drive_config = lusid_drive.Configuration(
    host = f'{api_instance.api_client.configuration.host[:-4]}/drive'
)
drive_config.access_token = api_factory.api_client.configuration.access_token

# Enter a context with an instance of the API client
with lusid_drive.ApiClient(drive_config) as api_client:
    # Create an instance of the API class
    api_instance = lusid_drive.FoldersApi(api_client)
    create_folder = {"path":"/","name":"finbourne university"} # CreateFolder | A CreateFolder object that defines the name and path of the new folder

    try:
        # [EARLY ACCESS] CreateFolder: Create a new folder in LUSID Drive
        api_response = api_instance.create_folder(create_folder)
        pprint(api_response)
    except ApiException as e:
        if not exception_guard(e, 664):
            print("Exception when calling FilesApi->create_file: %s\n" % e)
        
        
    api_instance = lusid_drive.FilesApi(api_client)
    x_lusid_drive_filename = 'instrument_subset.csv' # str | File name.
    x_lusid_drive_path = '/finbourne university/' # str | File path.
    content_length = len(subset_file.encode('UTF-8'))
    body = subset_file # str | 

    try:
        # [EARLY ACCESS] CreateFile: Uploads a file to Lusid Drive. If using an SDK, consider using the UploadAsStreamAsync function for larger files instead.
        api_response = api_instance.create_file(x_lusid_drive_filename, x_lusid_drive_path, content_length, body)
    except lusid_drive.rest.ApiException as e:
        if not exception_guard(e, 671):
            print("Exception when calling FilesApi->create_file: %s\n" % e)
        

# Luminesce Syntax

In this part of the course we will provide some additional technical information about luminesce, and demonstrate how luminesce queries can be used to extract information from your data.

## What is Structured Query Language (SQL)


SQL - pronounced "sequel" was first developed at IBM in the early 1970s, as a way to query and manipulate structured relational databases. SQL is a set-based, declarative programming language. Many implementations of SQL add extensions with procedural programming functionality, such as control-of-flow contructs.

## Luminesce Queries - SQL with extensions


SQLite is the most widely deployed database engine, used by several of the top web browsers, operating systems and application software.

Luminesce supports most of the query syntax for the SQLite dialect of SQL, but not the data manipulation or data definition syntax. Data can still be manipulated using some extensions to the language that Luminesce adds. 

Luminesce adds extensions to the SQLite implementation which can:
- exclude columns from queries
- automatically select important columns in queries
- parameterise queries
- write data to some Luminesce providers
- wait on results of some dependent operation, or for a time period
- run queries in parallel on an iterated range of parameters.



## Luminesce providers


A provider is a component that enables you to write a Luminesce SQL query for a data source in situ, without first having to extract, transform or load data from that source.

We supply providers for numerous data sources, including for the investment data stored in LUSID itself, for files stored in Drive and for some external data providers. You can also create your own provider for a proprietary data source.

There are two types of provider: a data provider and a direct provider.

A data provider is designed to query a data source whose shape is known. It therefore returns a table of results with a fixed number of fields (columns).

A direct provider is designed to query a data source whose shape is not known, and thus cannot return a table of results with a fixed number of fields (columns).

We'll use both data and direct providers in this tutorial.

## Use case: Using Luminesce to query and join data from disparate sources, and to distribute the resulting information.


I'd like to read a file from Drive and use the data from this file to filter transaction results. I'd like to join these filtered results with another provider and distribute the results. This will be packaged up into a simple to use View which runs on a schedule.

## An intro to SQLite queries

### A query using a simple SELECT statement


Its simple to get data from a provider using a select statement.

Use the following statement to get all data from a provider:

`SELECT * FROM SOME.PROVIDER`

Or retrieve data from a subset of columns by specifying them:

`SELECT A_COLUMN, ANOTHER_COLUMN FROM SOME.PROVIDER`

Here we'll use a SELECT statement to query for all of our porfolios.

In [8]:
%%luminesce
SELECT * 
FROM Lusid.Instrument.Equity
LIMIT 100

,LusidInstrumentId,Isin,Sedol,Cusip,Ticker,ClientInternal,Figi,CompositeFigi,ShareClassFigi,Wertpapier,...,Scope,DisplayName,LookThroughPortfolioScope,LookThroughPortfolioCode,Type,InferredAssetClass,InferredDomCcy,IsActive,Identifiers,DomCcy
0,LUID_00003DI9,US8825081040,NaN,882508104,NaN,NaN,BBG000BVV7G1,NaN,NaN,NaN,...,FBNUniversityModule-T2-2,Texas Instruments,NaN,NaN,Equity,Equities,USD,True,NaN,USD
1,LUID_00003D6S,GB0002374006,237400,NaN,NaN,NaN,BBG000BS69D5,NaN,NaN,NaN,...,default,Diageo,NaN,NaN,Equity,Equities,GBP,True,NaN,GBP
2,LUID_00003DH7,GB00B019KW72,B019KW7,NaN,NaN,NaN,BBG000BF0KW3,NaN,NaN,NaN,...,FBNUniversity,Sainsbury's,NaN,NaN,Equity,Equities,GBP,True,NaN,GBP
3,LUID_00003DH2,GB0033195214,3319521,NaN,NaN,NaN,BBG000BKH1W6,NaN,NaN,NaN,...,FBNUniversity,Kingfisher,NaN,NaN,Equity,Equities,GBP,True,NaN,GBP
4,LUID_00003DK7,US81181C1045,NaN,81181C104,NaN,NaN,BBG000BH0FR6,NaN,NaN,NaN,...,FBNUniversityModule-T2-2,Seagen Inc.,NaN,NaN,Equity,Equities,USD,True,NaN,USD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,LUID_00003D7S,US6092071058,NaN,609207105,NaN,NaN,BBG000D4LWF6,NaN,NaN,NaN,...,fbn-uni-T02002,Mondelēz International,NaN,NaN,Equity,Equities,USD,True,NaN,USD
96,LUID_00003D8Y,GB00B02L3W35,B02L3W3,NaN,NaN,NaN,BBG000H6ZKT3,NaN,NaN,NaN,...,fbn-uni-T02002,Berkeley Group Holdings,NaN,NaN,Equity,Equities,GBP,True,NaN,GBP
97,LUID_00003D5X,NaN,NaN,NaN,NaN,NaN,BBG000C05BD1,NaN,NaN,NaN,...,FBNUniversityModule-6-3,BP,NaN,NaN,Equity,Equities,GBP,True,NaN,GBP
98,LUID_00003DI6,GB00BDR05C01,BDR05C0,NaN,NaN,NaN,BBG000FV67Q4,NaN,NaN,NaN,...,FBNUniversity,National Grid plc,NaN,NaN,Equity,Equities,GBP,True,NaN,GBP


In this example, we've selected all columns and rows from the Lusid.Instrument.Equity provider. We can specify column names if we are only interested in some parts of the data.

In [9]:
%%luminesce
SELECT DisplayName, Figi 
FROM Lusid.Instrument.Equity
LIMIT 100

,DisplayName,Figi
0,Tesco,BBG000BF46Y8
1,Bunzl,BBG000BD7MX6
2,"Okta, Inc.",BBG001YV1SM4
3,BAE Systems,BBG000BD5TW4
4,Intel Corporation,BBG000C0G1D1
...,...,...
95,Croda International,BBG000BDCLS8
96,Trip.com Group,BBG000BDFGF0
97,"O'Reilly Automotive, Inc.",BBG000BGYWY6
98,Experian,BBG000BKFZN3


Here I really just want to see the name of the Equity and the Figi, I've selected only those columns from the provider.

### Limiting the number of return results using LIMIT


These simple SELECT statements are useful for getting an idea of the shape and quality of the data returned by a provider, but they tend to return a lot of data. Lets use LIMIT to reduce the amount of data requested:

In [10]:
%%luminesce
SELECT DisplayName, Figi 
FROM Lusid.Instrument.Equity 
LIMIT 5

,DisplayName,Figi
0,Texas Instruments,BBG000BVV7G1
1,Diageo,BBG000BS69D5
2,Sainsbury's,BBG000BF0KW3
2,"Training module 6.2 - Same transaction code, d...",Module-6-2
3,Kingfisher,BBG000BKH1W6
4,Seagen Inc.,BBG000BH0FR6


This example returns the first 5 rows of data from the Lusid.Instrument.Equity provider.

### Filter queries WHERE ...


We can now filter these results down - lets get all of the Equities that are listed in GBP


In [11]:
%%luminesce
SELECT * 
FROM Lusid.Instrument.Equity 
WHERE DomCcy = 'GBP'
LIMIT 100

,LusidInstrumentId,Isin,Sedol,Cusip,Ticker,ClientInternal,Figi,CompositeFigi,ShareClassFigi,Wertpapier,...,Scope,DisplayName,LookThroughPortfolioScope,LookThroughPortfolioCode,Type,InferredAssetClass,InferredDomCcy,IsActive,Identifiers,DomCcy
0,LUID_00003D8A,GB00BLGZ9862,BLGZ986,NaN,NaN,NaN,BBG000BF46Y8,NaN,NaN,NaN,...,fbn-uni-T02002,Tesco,NaN,NaN,Equity,Equities,GBP,True,NaN,GBP
1,LUID_00003D6L,GB00B0744B38,B0744B3,NaN,NaN,NaN,BBG000BD7MX6,NaN,NaN,NaN,...,default,Bunzl,NaN,NaN,Equity,Equities,GBP,True,NaN,GBP
2,LUID_00003DD5,GB0002634946,263494,NaN,NaN,NaN,BBG000BD5TW4,NaN,NaN,NaN,...,FBNUniversity,BAE Systems,NaN,NaN,Equity,Equities,GBP,True,NaN,GBP
3,LUID_00003DC7,GB00BDR05C01,BDR05C0,NaN,NaN,NaN,BBG000FV67Q4,NaN,NaN,NaN,...,fbn-uni-T02002,National Grid plc,NaN,NaN,Equity,Equities,GBP,True,NaN,GBP
4,LUID_00003DLS,GB00B019KW72,B019KW7,NaN,NaN,NaN,BBG000BF0KW3,NaN,NaN,NaN,...,FBNUniversityModule-T2-2,Sainsbury's,NaN,NaN,Equity,Equities,GBP,True,NaN,GBP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,LUID_00003D7P,ES0177542018,B5M6XQ7,NaN,NaN,NaN,BBG000C04GS5,NaN,NaN,NaN,...,fbn-uni-T02002,International Airlines Group,NaN,NaN,Equity,Equities,GBP,True,NaN,GBP
96,LUID_00003D8H,JE00B4T3BW64,B4T3BW6,NaN,NaN,NaN,BBG001MM1KV4,NaN,NaN,NaN,...,fbn-uni-T02002,Glencore,NaN,NaN,Equity,Equities,GBP,True,NaN,GBP
97,LUID_00003DLC,GB00B1FH8J72,B1FH8J7,NaN,NaN,NaN,BBG000BS5FD2,NaN,NaN,NaN,...,FBNUniversityModule-T2-2,Severn Trent,NaN,NaN,Equity,Equities,GBP,True,NaN,GBP
98,LUID_00003DNO,GB00BKKMKR23,BKKMKR2,NaN,NaN,NaN,BBG000H3DN90,NaN,NaN,NaN,...,FBNUniversityModule-T2-2,RSA Insurance Group,NaN,NaN,Equity,Equities,GBP,True,NaN,GBP


We've filtered our query for equities who have a domestic currency of Pound Sterling.

### JOINing disparate datasources with join


Now, I've got a file in drive with sector information for some of these instruments. I'm going to join data from the file using our Drive provider with the saved instrument data so I can see the sectors for instruments where we know this information.

First lets get our instrument data.

Then we'll grab the file contents from drive.

We'll take a look at cross-joins, which SQLite uses to build left joins and inner joins. Cross-joins return the cartesian product of both tables - each row on the left table is concatenated with every row on the right table, to form a massive table with every combination of both tables' rows:


In [12]:
%%luminesce

/*
    We'll cover using variables later in the tutorial
*/

@x = use Drive.Csv
--file=/finbourne university/instrument_subset.csv
enduse;

SELECT DisplayName, DomCcy, File.Sector 
FROM Lusid.Instrument.Equity as Equities 
CROSS JOIN @x as File
LIMIT 100

,DisplayName,DomCcy,Sector
0,Texas Instruments,USD,Financial Services
1,Texas Instruments,USD,General Industrials
2,Texas Instruments,USD,Travel & Leisure
3,Texas Instruments,USD,Equity Investment Instruments
4,Texas Instruments,USD,Household Goods & Home Construction
...,...,...,...
95,Kingfisher,GBP,Mining
96,Kingfisher,GBP,Media
97,Kingfisher,GBP,Mobile Telecommunications
98,Kingfisher,GBP,Food & Drug Retailers


We've returned a table with every combination of Lusid Equities with every row in our file.

In a left join, all rows from the left table are returned, along with rows in the right table that match the supplied key constraint. If there is a row in the left table with no matching row on the right, the columns that come from the right table in the resulting row are set to null.

In [13]:
%%luminesce

/*
    We'll cover using variables later in the tutorial
*/

@x = use Drive.Csv
--file=/finbourne university/instrument_subset.csv
enduse;

SELECT DisplayName, DomCcy, File.Sector, Equities.Figi as Figi 
FROM Lusid.Instrument.Equity as Equities 
LEFT JOIN @x as File on Equities.Figi = File.Figi
LIMIT 100

,DisplayName,DomCcy,Sector,Figi
0,Tesco,GBP,NaN,BBG000BF46Y8
1,Bunzl,GBP,NaN,BBG000BD7MX6
2,"Okta, Inc.",USD,NaN,BBG001YV1SM4
3,BAE Systems,GBP,NaN,BBG000BD5TW4
4,Intel Corporation,USD,NaN,BBG000C0G1D1
...,...,...,...,...
95,Croda International,GBP,NaN,BBG000BDCLS8
96,Trip.com Group,USD,NaN,BBG000BDFGF0
97,"O'Reilly Automotive, Inc.",USD,NaN,BBG000BGYWY6
98,Experian,GBP,Support Services,BBG000BKFZN3


We see this here as there are some companies with no matching sector in our file - these are returned as null.

We are going to ignore instruments without a matching sector for now, so we'll use an inner join.
Inner joins only return rows where the joining key has a match on both sides of the table:

In [14]:
%%luminesce

/*
    We'll cover using variables later in the tutorial
*/

@x = use Drive.Csv
--file=/finbourne university/instrument_subset.csv
enduse;

SELECT DisplayName, DomCcy, File.Sector, Equities.Figi as Figi 
FROM Lusid.Instrument.Equity as Equities 
INNER JOIN @x as File on Equities.Figi = File.Figi
LIMIT 100

,DisplayName,DomCcy,Sector,Figi
0,Aviva,GBP,Life Insurance,BBG000DDCSJ1
1,BP,GBP,Real Estate Investment Trusts,BBG000BDPHS3
2,Ocado Group,GBP,Food & Drug Retailers,BBG000BDXZH6
3,Land Securities,GBP,Real Estate Investment Trusts,BBG000BDPHS3
4,BP,GBP,Oil & Gas Producers,BBG000C05BD1
...,...,...,...,...
90,Vodafone Group,GBP,Mobile Telecommunications,BBG000C6K6G9
91,Taylor Wimpey,GBP,Household Goods & Home Construction,BBG000BF4KL1
92,Ferguson plc,GBP,Support Services,BBG00NZJ0JG0
93,British Land,GBP,Real Estate Investment Trusts,BBG000BD7DW7


Now we have a joined table where only Equities with a sector on file are returned, along with their sector.

### GROUPing results BY some condition in query


We can also group our results by a column - we'll perform an aggregation and group-by to count the number of instrument in each sector.

There are many different aggregation functions in SQLite, such as sum, avg and count.

Sum will return the sum of the specified field in each group.
Average returns the mean of the specified field in each group.
Count returns the number of results that are non-null for the field in each group. We can also specify that count returns the count of distinct values [Count(Distinct <field>)], or all rows [count(*)], in each group.

In [15]:
%%luminesce

/*
    We'll cover using variables later in the tutorial
*/

@x = use Drive.Csv
--file=/finbourne university/instrument_subset.csv
enduse;

SELECT count(File.Sector), File.Sector
FROM Lusid.Instrument.Equity as Equities 
INNER JOIN @x as File on Equities.Figi = File.Figi
GROUP BY File.Sector
LIMIT 100

,count(File_Sector COLLATE BINARY),Sector
0,3,Automobiles & Parts
1,3,Equity Investment Instruments
2,6,Financial Services
3,3,Food & Drug Retailers
4,3,General Industrials
5,4,Household Goods & Home Construction
6,3,Investment Services
7,13,Life Insurance
8,3,Media
9,9,Mining


We've now grouped our results by sector, and for each sector, we''ve returned how many Equities belong to the sector.

## Introducing some Luminesce extensions


### Using ^ to SELECT the most important columns from a provider


Let's use the carat symbol to grab then more important columns in the Equity provider. The list of important columns is decided by the writer of the Provider.

In [16]:
%%luminesce
SELECT ^ 
FROM Lusid.Instrument.Equity
LIMIT 100

,LusidInstrumentId,Scope,DisplayName,Type,InferredAssetClass,InferredDomCcy,IsActive
0,LUID_00003DI9,FBNUniversityModule-T2-2,Texas Instruments,Equity,Equities,USD,True
1,LUID_00003D6S,default,Diageo,Equity,Equities,USD,True
2,LUID_00003DH7,FBNUniversity,Sainsbury's,Equity,Equities,GBP,True
1,LUID_00003D6L,default,Bunzl,Equity,Equities,GBP,True
2,LUID_00003DDV,FBNUniversity,"Okta, Inc.",Equity,Equities,USD,True
3,LUID_00003DH2,FBNUniversity,Kingfisher,Equity,Equities,GBP,True
4,LUID_00003DK7,FBNUniversityModule-T2-2,Seagen Inc.,Equity,Equities,USD,True
...,...,...,...,...,...,...,...
95,LUID_00003D7S,fbn-uni-T02002,Mondelēz International,Equity,Equities,GBP,True
96,LUID_00003D8Y,fbn-uni-T02002,Berkeley Group Holdings,Equity,Equities,GBP,True


In the Lusid Equities provider: LusidInstrumentId, Scope, DisplayName, InferredAssetClass, InferredDomCcy and IsActive are the fields marked by the provider as being most important.

### Excluding columns from the results of a query using the except keyword.


Using some providers, we can also exclude columns from the result of a query by using the except keyword:

In [17]:
%%luminesce
SELECT ^ (except Type, IsActive) 
FROM Lusid.Instrument.Equity
LIMIT 100

,LusidInstrumentId,Scope,DisplayName,InferredAssetClass,InferredDomCcy
0,LUID_00003D8A,fbn-uni-T02002,Tesco,Equities,GBP
1,LUID_00003D6L,default,Bunzl,Equities,GBP
2,LUID_00003DDV,FBNUniversity,"Okta, Inc.",Equities,USD
3,LUID_00003DD5,FBNUniversity,BAE Systems,Equities,GBP
4,LUID_00003DAK,fbn-uni-T02002,Intel Corporation,Equities,USD
...,...,...,...,...,...
95,LUID_00003D54,FBNUniversityModule-1,Croda International,Equities,USD
96,LUID_00003D8R,fbn-uni-T02002,Trip.com Group,Equities,USD
97,LUID_00003DDI,FBNUniversity,"O'Reilly Automotive, Inc.",Equities,USD
98,LUID_00003DCL,FBNUniversity,Experian,Equities,GBP


Here we select the most important columns again, but exclude the Type and IsActive columns

### Using parameters to filter queries


Many providers will have parameters that can also be used to filter a Luminesce query as part of a standard WHERE clause.

We can see these parameters by querying the Sys.Field Provider.

In [18]:
%%luminesce
SELECT FieldName, DataType, ParamDefaultValue, Description 
FROM Sys.Field 
WHERE TableName = 'Lusid.Instrument.Equity' 
AND FieldType = 'Parameter'
LIMIT 100

,FieldName,DataType,ParamDefaultValue,Description
0,AsAt,DateTime,0001-01-01T00:00:00Z,As at time to query at. Defaults to latest.
1,EffectiveAt,DateTime,0001-01-01T00:00:00Z,Effective time to query at. Defaults to latest.
2,UseLusidFilter,Boolean,True,Should the filter be translated to a Finbourne...


We can see there are 3 fields we can filter our query on: AsAt, EffectiveAt and UseLusidFilter

In [19]:
%%luminesce

SELECT * 
FROM Lusid.Instrument.Equity
WHERE UseLusidFilter = False
LIMIT 100

LusidInstrumentId          Isin    Sedol      Cusip Ticker ClientInternal  \
0      LUID_00003DI9  US8825081040      NaN  882508104    NaN            NaN   
1      LUID_00003D6S  GB0002374006   237400        NaN    NaN            NaN   
2      LUID_00003DH7  GB00B019KW72  B019KW7        NaN    NaN            NaN   
3      LUID_00003DH2  GB0033195214  3319521        NaN    NaN            NaN   
4      LUID_00003DK7  US81181C1045      NaN  81181C104    NaN            NaN   
..               ...           ...      ...        ...    ...            ...   
95     LUID_00003D7S  US6092071058      NaN  609207105    NaN            NaN   
96     LUID_00003D8Y  GB00B02L3W35  B02L3W3        NaN    NaN            NaN   
97     LUID_00003D5X           NaN      NaN        NaN    NaN            NaN   
98     LUID_00003DI6  GB00BDR05C01  BDR05C0        NaN    NaN            NaN   
99     LUID_00003DFO  GB0006043169   604316        NaN    NaN            NaN   

            Figi  CompositeFigi  ShareClassFigi  Wertpapier  ...  \
0   BBG000BVV7G1            NaN             NaN         NaN  ...   
1   BBG000BS69D5            NaN             NaN         NaN  ...   
2   BBG000BF0KW3            NaN             NaN         NaN  ...   
3   BBG000BKH1W6            NaN             NaN         NaN  ...   
4   BBG000BH0FR6            NaN             NaN         NaN  ...   
..           ...            ...             ...         ...  ...   
95  BBG000D4LWF6            NaN             NaN         NaN  ...   
96  BBG000H6ZKT3            NaN             NaN         NaN  ...   
97  BBG000C05BD1            NaN             NaN         NaN  ...   
98  BBG000FV67Q4            NaN             NaN         NaN  ...   
99  BBG000GVNZ28            NaN             NaN         NaN  ...   

                       Scope              DisplayName  \
0   FBNUniversityModule-T2-2        Texas Instruments   
1                    default                   Diageo   
2              FBNUniversity              Sainsbury's   
3              FBNUniversity               Kingfisher   
4   FBNUniversityModule-T2-2              Seagen Inc.   
..                       ...                      ...   
95            fbn-uni-T02002   Mondelēz International   
96            fbn-uni-T02002  Berkeley Group Holdings   
97   FBNUniversityModule-6-3                       BP   
98             FBNUniversity        National Grid plc   
99             FBNUniversity                Morrisons   

    LookThroughPortfolioScope LookThroughPortfolioCode    Type  \
0                         NaN                      NaN  Equity   
1                         NaN                      NaN  Equity   
2                         NaN                      NaN  Equity   
3                         NaN                      NaN  Equity   
4                         NaN                      NaN  Equity   
..                        ...                      ...     ...   
95                        NaN                      NaN  Equity   
96                        NaN                      NaN  Equity   
97                        NaN                      NaN  Equity   
98                        NaN                      NaN  Equity   
99                        NaN                      NaN  Equity   

    InferredAssetClass  InferredDomCcy IsActive Identifiers DomCcy  
0             Equities             USD     True         NaN    USD  
1             Equities             GBP     True         NaN    GBP  
2             Equities             GBP     True         NaN    GBP  
3             Equities             GBP     True         NaN    GBP  
4             Equities             USD     True         NaN    USD  
..                 ...             ...      ...         ...    ...  
95            Equities             USD     True         NaN    USD  
96            Equities             GBP     True         NaN    GBP  
97            Equities             GBP     True         NaN    GBP  
98            Equities             GBP     True         N

Here we've filtered an Equity query on the UseLusidFilter. Notice there is no returned Field called UseLusidFilter - its not a column in our data.

## Using Luminesce variables


You can use variables to create and populate arbitrary tables of data that can then be used as part of a Luminesce query for either a data provider or a direct provider.

Declare a variable that represents a table of data using:
`@variable`

Or to represent a single value:
`@@variable`

Let's see variables in action. 

In [20]:
%%luminesce
@instruments_in_GBP = SELECT * 
                      FROM Lusid.Instrument.Equity 
                      WHERE DomCcy = 'GBP';
SELECT Isin 
FROM @instruments_in_GBP
LIMIT 100

,Isin
0,GB00BLGZ9862
1,GB00B0744B38
2,GB0002634946
3,GB00BDR05C01
4,GB00B019KW72
...,...
94,ES0177542018
95,JE00B4T3BW64
96,GB00B1FH8J72
97,GB00BKKMKR23


Here_instruments_in GBP is a table variable, set from the results of the query on equities.

In [21]:
%%luminesce
@sectors = use Drive.Csv
--file=/finbourne university/instrument_subset.csv
enduse;

@@selected_sector = SELECT 'Life Insurance';
SELECT * 
FROM @sectors 
WHERE Sector = @@selected_sector
LIMIT 100

,Name,Sector,Figi
0,Prudential plc,Life Insurance,BBG000BDY322
1,St. James's Place plc,Life Insurance,BBG000BRMPC4
2,Legal & General,Life Insurance,BBG000BDQCG6
3,Aviva,Life Insurance,BBG000DDCSJ1


I've set selected_sector as a single-value variable "Lise Insurance", and filtered my file by sector.

## Introducing more advanced Luminesce extensions


You can use the CROSS APPLY or OUTER APPLY statement to iterate over a set of parameters. This is useful to execute a Luminesce query in parallel for each of the parameter inputs.

### Cross-apply

Cross-apply is similar to INNER JOIN, in that only records that have matching values on both sides are returned. 
We'll run some equity queries in parallel using Tools.split to create a set of currencies to filter by- the table will contain the currency joined to all of the returned equity tables:

In [22]:
%%luminesce

select VALUE as val
from Tools.Split 
where Original = 'GBP USD CHF' 
AND Delimiters = ' ';


,val
0,GBP
1,USD
2,CHF


First, we use Tools.Split to turn a space separated string of currencies into into a table where each currency is an entry in the table.

In [23]:
%%luminesce

@currencies = select VALUE as val
              from Tools.Split 
              where Original = 'GBP USD CHF' 
              AND Delimiters = ' ';

SELECT c.val as selectedCurrency, results.*
FROM @currencies c
CROSS APPLY (
       SELECT equity.^ 
       FROM Lusid.Instrument.Equity equity
       WHERE equity.DomCcy = c.val
       
) results
LIMIT 100

,selectedCurrency,LusidInstrumentId,Scope,DisplayName,Type,InferredAssetClass,InferredDomCcy,IsActive,__row_key__
0,GBP,LUID_00003D6S,default,Diageo,Equity,Equities,GBP,True,1
1,GBP,LUID_00003DH7,FBNUniversity,Sainsbury's,Equity,Equities,GBP,True,1
2,GBP,LUID_00003DH2,FBNUniversity,Kingfisher,Equity,Equities,GBP,True,1
3,GBP,LUID_00003D6A,default,Aviva,Equity,Equities,GBP,True,1
4,GBP,LUID_00003D56,FBNUniversityModule-1,BP,Equity,Equities,GBP,True,1
...,...,...,...,...,...,...,...,...,...
95,GBP,LUID_00003D6B,default,B&M,Equity,Equities,GBP,True,1
96,GBP,LUID_00003DCG,fbn-uni-T02002,Admiral Group,Equity,Equities,GBP,True,1
97,GBP,LUID_00003DEC,FBNUniversity,United Utilities,Equity,Equities,GBP,True,1
98,GBP,LUID_00003DMD,FBNUniversityModule-T2-2,Halma,Equity,Equities,GBP,True,1


We then save the result of that query in a variable @currency, which is passed as a criterion in the CROSS APPLY subquery,  which runs in parallel for each row in @currency.

This example works similar to an inner join, we've run a query in parallel for some currency values, returning a similar table to an inner join. Remember, inner join returns only rows which fulfil the join criteria.

### Outer-apply


Outer apply is similar to LEFT OUTER JOIN, in that all records on the left side are returned, even when there are no matching records on the right side.

In [23]:
%%luminesce

@currencies = select VALUE as val
              from Tools.Split 
              where Original = 'GBP USD CHF' 
              AND Delimiters = ' ';
              
SELECT c.val, results.*
FROM @currencies c
OUTER APPLY (
       SELECT equity.^ 
       FROM Lusid.Instrument.Equity as equity
       WHERE equity.DomCcy = c.val
) results
LIMIT 100

,val,LusidInstrumentId,Scope,DisplayName,Type,InferredAssetClass,InferredDomCcy,IsActive,__row_key__
0,GBP,LUID_00003D4V,FBNUniversityModule-1,RELX,Equity,Equities,GBP,True,1
1,GBP,LUID_00003D4W,FBNUniversityModule-1,Halma,Equity,Equities,GBP,True,1
2,GBP,LUID_00003D4X,FBNUniversityModule-1,Unilever,Equity,Equities,GBP,True,1
3,GBP,LUID_00003D4Y,FBNUniversityModule-1,Just Eat Takeawy,Equity,Equities,GBP,True,1
4,GBP,LUID_00003D4Z,FBNUniversityModule-1,Mondi,Equity,Equities,GBP,True,1
...,...,...,...,...,...,...,...,...,...
95,GBP,LUID_00003D8S,fbn-uni-T02002,Fresnillo,Equity,Equities,GBP,True,1
96,GBP,LUID_00003D8W,fbn-uni-T02002,Royal Dutch Shell,Equity,Equities,GBP,True,1
97,GBP,LUID_00003D8X,fbn-uni-T02002,Compass Group,Equity,Equities,GBP,True,1
98,GBP,LUID_00003D8Y,fbn-uni-T02002,Berkeley Group Holdings,Equity,Equities,GBP,True,1


We've run a query in parallel for some currency values, returning a similar table to a left join with our selected currencies on the left side of the join. Here, the values from Lusid.Instrument.Equity on the CHF row are null.